# Preprocessing US Census Education Data

Data was downloaded from https://data.census.gov/table/ACSDT5Y2023.B15003?q=B15003&g=010XX00US$5000000 and saved to input_path.

US district population educational attainment info from 2011-2023.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
input_path = '/content/drive/MyDrive/ANLP Project/Final_Version/Data/US_Census_Education_By_District/'
output_path = '/content/drive/MyDrive/ANLP Project/Final_Version/Data/'

Mounted at /content/drive


In [ ]:
# imports
import pandas as pd
import glob

The original dataset provides district population, number of people with bachelor's, number of people with masters, number of people with professional school degrees, and number of people with doctorates. For this project, higher education will be considered bachelors or higher.

Final saved df will contain `NAME`, `total_population`, `bachelors`, `masters`, `professional_school`, `doctorate`, `year`, `state`, `district`, `bachelors_or_higher_percentage`.

In [ ]:
df_list = []
years = range(2012, 2023+1)
numeric_cols = ['B15003_001E', 'B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E'] # total, bachelors, masters, professional school, doctorate
cols = ['NAME'] + numeric_cols

for year in years:
  # read all CSV files in the folder
  df = pd.read_csv(input_path + f'ACSDT5Y{year}.B15003-Data.csv')
  df = df[cols][1:-1] # remove first row col descriptions and last row puerto rico
  for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
  df['year'] = year

  df = df.rename(columns={
      "B15003_001E": "total_population",
      "B15003_022E": "bachelors",
      "B15003_023E": "masters",
      "B15003_024E": "professional_school",
      "B15003_025E": "doctorate"
  })

  # extract state (everything after the final comma)
  df["state"] = df["NAME"].str.extract(r",\s*(.*)$")

  # extract district information
  df["NAME"] = df["NAME"].str.replace(r'\(at large\)', '1', case=False, regex=True)
  df["district"] = df["NAME"].str.extract(r"Congressional District\s*(.*?)\s*\(")
  df = df[df['district'] != 's not defined'] # population is 0 anyway
  df['district'] = pd.to_numeric(df['district']).astype('Int64')

  # get educated % (all degrees bachelor and higher / total population)
  df['bachelors_or_higher_percentage'] = (df['bachelors'] + df['masters'] + df['professional_school'] + df['doctorate']) / (df['total_population'])

  df_list.append(df)

df = pd.concat(df_list, ignore_index=True)
df = df[df['state'] != 'District of Columbia'] # not included in 435 house of reps

# check result
print(df.shape)
df.head(20)

(5220, 10)


,NAME,total_population,bachelors,masters,professional_school,doctorate,year,state,district,bachelors_or_higher_percentage
0,"Congressional District 1 (113th Congress), Ala...",454022,63944,22780,5781,3915,2012,Alabama,1,0.212369
1,"Congressional District 2 (113th Congress), Ala...",452532,60046,25794,5466,3149,2012,Alabama,2,0.208726
2,"Congressional District 3 (113th Congress), Ala...",445285,51775,22711,4594,5316,2012,Alabama,3,0.189533
3,"Congressional District 4 (113th Congress), Ala...",462881,42138,18364,5276,2471,2012,Alabama,4,0.147444
4,"Congressional District 5 (113th Congress), Ala...",458812,83168,35703,6405,5199,2012,Alabama,5,0.284376
5,"Congressional District 6 (113th Congress), Ala...",460127,98381,37542,12850,6240,2012,Alabama,6,0.336892
6,"Congressional District 7 (113th Congress), Ala...",432765,48665,20066,4290,2972,2012,Alabama,7,0.175599
7,"Congressional District 1 (113th Congress), Alaska",447543,79441,30708,7497,5322,2012,Alaska,1,0.274762
8,"Congressional District 1 (113th Congress), Ari...",447385,62724,28867,5399,5357,2012,Arizona,1,0.228767
9,"Congressional District 2 (113th Congress), Ari...",493408,91440,44016,10567,9287,2012,Arizona,2,0.31477


In [ ]:
# save csv
df.to_csv(output_path + 'us_census_education_by_district_cleaned.csv')